# **Set according to environment (e.g. local, Google Colab...)**

In [1]:
project_folder = ''

# **Body**

In [2]:
from custom_libraries.image_dataset import *
from custom_libraries.miscellaneous import *
import numpy as np

In [3]:
def create_model(num_units):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.Dense(units=num_units, activation=None, kernel_initializer=tf.keras.initializers.HeNormal, ))
    model.add(tf.keras.layers.LeakyReLU(alpha=.01))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[tf.keras.losses.BinaryCrossentropy(name='binary_crossentropy'), 'acc'])

    return model

In [4]:
bs = 256
trials = 10
epochs = 2000
trees_set = [1, 32]

classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)
# TODO: verificare l'effetto di patience sul risultato di svhn (rischio underfitting)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#classes = [classes[-2]]
history = np.zeros((len(classes), trials, len(trees_set), 2))

#history = np.load(project_folder+'results/fcnn_history.npy', allow_pickle=True)
#history[-2,:,:,:] = 0

for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    #if history[j, 0, 0, 0] != 0:
    #    continue

    test_ds = ImageDataset(ds, 'test', data_dir=None, USPS_dir=project_folder + 'USPS/')
    train_ds = ImageDataset(ds, 'train', data_dir=None, USPS_dir=project_folder + 'USPS/')

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                         validation_size=len(test_ds.images))
    test_set = tf.data.Dataset.from_tensor_slices((test_ds.images, test_ds.labels)).batch(bs)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")

        for i in range(trials):

            if history[j, i, k, 0] != 0:
                continue

            print(f"Trial {i + 1}")

            model = create_model(num_units=2 * trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).batch(bs)

            with tf.device('/device:GPU:0'):

                fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                        validation_data=valid_set, validation_batch_size=bs,
                                        callbacks=[callback], verbose=0)
                print_fit_history(fit_history, epochs)

                evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
                print_evaluate_history(evaluate_history)

                history[j, i, k] = evaluate_history[1:]

                np.save(project_folder + 'results/fcnn_history.npy', history,
                        allow_pickle=True)

Dataset: mnist / Pair: 3-5
1-FCNN
Trial 1
Epochs: 168/2000 - Train BCE: 0.0454, accuracy: 99.27% - Validation BCE: 0.1637, accuracy: 94.24%
Test BCE: 0.1232, accuracy: 95.27%
Trial 2
Epochs: 669/2000 - Train BCE: 0.0511, accuracy: 99.9% - Validation BCE: 0.2477, accuracy: 93.72%
Test BCE: 0.1355, accuracy: 96.74%
Trial 3
Epochs: 119/2000 - Train BCE: 0.0546, accuracy: 98.96% - Validation BCE: 0.1783, accuracy: 93.68%
Test BCE: 0.1186, accuracy: 95.16%
Trial 4
Epochs: 160/2000 - Train BCE: 0.0532, accuracy: 98.45% - Validation BCE: 0.1778, accuracy: 93.68%
Test BCE: 0.1155, accuracy: 95.06%
Trial 5
Epochs: 181/2000 - Train BCE: 0.0503, accuracy: 99.38% - Validation BCE: 0.1717, accuracy: 93.68%
Test BCE: 0.1286, accuracy: 95.69%
Trial 6
Epochs: 432/2000 - Train BCE: 0.0579, accuracy: 99.9% - Validation BCE: 0.2096, accuracy: 95.26%
Test BCE: 0.1681, accuracy: 94.85%
Trial 7
Epochs: 131/2000 - Train BCE: 0.0615, accuracy: 98.76% - Validation BCE: 0.1926, accuracy: 93.68%
Test BCE: 0.1298

In [5]:
history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{k}-FCNN")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")

[[[[0.12315002 0.95268136]
   [0.10301321 0.96424818]]

  [[0.1355453  0.96740276]
   [0.08180099 0.97371191]]

  [[0.11859088 0.95162988]
   [0.08577724 0.97055733]]

  [[0.11552219 0.95057833]
   [0.06699809 0.97423762]]

  [[0.1286492  0.95688748]
   [0.11239141 0.96740276]]

  [[0.1680911  0.9484753 ]
   [0.09763914 0.96582544]]

  [[0.12977189 0.95268136]
   [0.11707418 0.96319664]]

  [[0.12605593 0.95478445]
   [0.18187484 0.96845424]]

  [[0.13425477 0.95320714]
   [0.11900771 0.95899051]]

  [[0.14042614 0.95636171]
   [0.12713566 0.96372241]]]


 [[[0.44353211 0.83350003]
   [0.4947983  0.82349998]]

  [[0.38302454 0.82800001]
   [0.4369607  0.81950003]]

  [[0.41702592 0.82849997]
   [0.4761976  0.8265    ]]

  [[0.39608443 0.83950001]
   [0.44002607 0.83099997]]

  [[0.42842469 0.82300001]
   [0.53747529 0.82550001]]

  [[0.37253889 0.82999998]
   [0.44984889 0.82849997]]

  [[0.36945379 0.8405    ]
   [0.43368852 0.83649999]]

  [[0.4082841  0.838     ]
   [0.4420003  0.83